# MSiA 420 - Predictive Analytics II - Final Project
## Group 6: Alejandra Lelo de Larrea Ibarra, Kiran Jyothi Sheena, Lixuan (Ellen) Chen, Wencheng Zhang

# Cleaning data

In [1]:
# Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sns

In [52]:
# Read data
data = pd.read_csv("../02_Data/hotel_bookings.csv")
country = pd.read_csv("../02_Data/country_mapping.csv")

# Dimensions 
print("Dimensions: " + str(data.shape))

# Print header 
data.head()

Dimensions: (119390, 32)


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [56]:
# --- CLEAN DATA ---
# Create clean dataframe as a copy of original data 
data_clean = data.copy()

# Create numerical column (arrival_month) representing the month
month_dict = {'January':1,'February':2,'March':3,'April':4,'May':5,'June':6,'July':7,'August':8,'September':9,
              'October':10,'November':11,'December':12}
#data_clean['arrival_month'] = data_clean['arrival_date_month']
data_clean['arrival_month'] = data_clean['arrival_date_month'].replace(month_dict)


# Create arrival_date column 
# Note: it might not be needed for regression, could be dropped later.
data_clean['arrival_date'] =data_clean['arrival_date_year'].astype('str') + "-" + \
                            data_clean['arrival_month'].astype('str') + "-" + \
                            data_clean['arrival_date_day_of_month'].astype('str')   
data_clean['arrival_date'] = pd.to_datetime(data_clean['arrival_date'])


# lead_time: take logs. -> log_lead_time
data_clean['log_lead_time'] = np.log(data_clean['lead_time']+1)

# Create a totalnights stayed 
data_clean['total_nights'] = data_clean['stays_in_weekend_nights'] + data_clean['stays_in_week_nights']

# Create total_nights_bucket for total nights stayed
night_bins = [0,6,11,16,21]
night_labels = ['A. 0-5','B. 6-10','C. 11-15','D. 16-20']
data_clean['total_nights_bucket'] = pd.cut(data_clean['total_nights'],bins= night_bins,labels=night_labels,
                                             right=False,include_lowest=True)
data_clean['total_nights_bucket'] = data_clean['total_nights_bucket'].astype('str')
data_clean['total_nights_bucket'] = data_clean['total_nights_bucket'].replace({'nan':'E. > 20'})

# ELLEN'S CLEANING HERE

# Avoiding chained assignment warnings
mine = data_clean.loc[:,['stays_in_week_nights', 'adults', 'children', 'babies', 
                         'meal', 'country', 'market_segment', 'distribution_channel']]

# code booking with missing children values as 0 children
data_clean['children'] = mine['children'].fillna(value = 0)

# adding a column "domestic" from counrty, if guest from Portugal, they are domestic
# Here bookings with unknown orgin are treated as not domestic (488 bookings)
data_clean['domestic'] = np.where(
    mine['country'] == 'PRT', 'domestic', 'international')

# Using a found country(ISO3166)-continent mapping table to assign continent/regions for the guests
# Source: https://github.com/lukes/ISO-3166-Countries-with-Regional-Codes/blob/master/all/all.csv
data_clean = data_clean.merge(country[['alpha-3','region', 'sub-region']],
           how = 'left', left_on = 'country', right_on = 'alpha-3').drop(columns = 'alpha-3')

# rename region to continent, sub-region to sub_continent
data_clean = data_clean.rename(columns = {'region':'continent', 'sub-region':'sub_continent'})

# Some countries were not mapped using the table from the online source
data_clean[data_clean.continent.isna()].country.unique()

# Hard-coding these continents, CN is Asia-East Asia, TMP is Asia-Southest Asia, ATA is Antarctica-Antarctica
# China
data_clean['continent'], data_clean['sub_continent']  = np.where(
    (mine['country'] == 'CN'), "Asia", data_clean['continent']), np.where(
    (mine['country'] == 'CN'), "Eastern Asia", data_clean['sub_continent'])
# East Timor
data_clean['continent'], data_clean['sub_continent']  = np.where(
    (mine['country'] == 'TMP'), "Asia", data_clean['continent']), np.where(
    (mine['country'] == 'TMP'), "South-eastern Asia", data_clean['sub_continent'])
# Antarctica
data_clean['continent'], data_clean['sub_continent']  = np.where(
    (mine['country'] == 'ATA'), "Antarctica", data_clean['continent']), np.where(
    (mine['country'] == 'ATA'), "Antarctica", data_clean['sub_continent'])
# Missing Values
data_clean['continent'], data_clean['sub_continent'] = data_clean['continent'].fillna(value = 'unknown'), \
                                                            data_clean['sub_continent'].fillna(value = 'unknown')

# Wencheng's cleaning
#how many you book before = previous cancellation + previous not cancellation
data_clean["previous_bookings"] = data_clean["previous_cancellations"]+data_clean["previous_bookings_not_canceled"]

#whether you get the same room: 1=you get you booked room, 0=no
data_clean["got_room_booked"]  = np.where(data_clean['reserved_room_type'] == data_clean['assigned_room_type'],1,0)

#agent_new = replace  nan with no-agent
data_clean["agent_new"] = np.where(data_clean["agent"].isna(),"no-agent",data_clean["agent"])

#agent: change to dummy -> booked_by_agent = 1 (if yes), 0 (if no)
#data_clean["booked_by_agent"] = np.where(data_clean['agent'].isna(),0,1)
data_clean["booked_by_agent"] = np.where(data_clean['agent'].isna(),'yes','no')

# Group the bookings by the agent and calculate the total number of cancellations
agent_cancellations = data_clean.groupby('agent').sum()
# Create a dictionary to map the agent to its corresponding risk bin
bins = [0,1895, 5685, 13264] #the max cancellation for agent is 13264
labels = ['1', '2', '3']#we group three bin to make sure that each bin share the rough balancen. 1=less cancel,2=moderate cancel,3=heavy cancel
agent_risk_dict = dict(zip(agent_cancellations.index, pd.cut(agent_cancellations["is_canceled"], bins=bins, labels=labels)))
data_clean["agent_risk"] = data_clean["agent"].map(agent_risk_dict)
data_clean["agent_risk"] = pd.to_numeric(np.where(data_clean["agent_risk"].isna(),0,data_clean["agent_risk"])) # if no agent, then agent_risk = 0
#we group the agent into different level based on cancellation rate, but there are some agent (304) whose the booking is never cancelled
#therefore, we regard null values and agent that is never cancelling as a unclassified

# Kiran's cleaning
# company: change to dummy -> booked_by_company = 1 (if yes), 0 (if no)
data_clean["booked_by_company"] = np.where(data_clean['company'].isna(), 0, 1)

# days_in_waiting_list: take logs. -> log_days_in_waiting_list.
# NOTE: we could go back and remove outliers. 
data_clean["log_days_in_waiting_list"] = np.log(data_clean["days_in_waiting_list"]+1)

# adr: take logs, we find that 5400 and -6.36 are outliers, so we remove them 
data_clean = data_clean[data_clean["adr"]>= 0] # remove negative prices
data_clean = data_clean[data_clean["adr"] != 5400] # remove big price 
data_clean["log_adr"] = np.log(data_clean.adr+1)

# required_car_parking_spaces: change to dummy -> required_car_parking = 1 (if yes) 0 (if no)
data_clean["required_car_parking"] = np.where(data_clean['required_car_parking_spaces']>0, 1, 0)

In [43]:
data_clean

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,sub_continent,total_booked,got_room_booked,agent_new,booked_by_agent,agent_risk,booked_by_company,log_days_in_waiting_list,log_adr,required_car_parking
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,Southern Europe,0,1,no-agent,yes,0,0,0.0,0.000000,0
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,Southern Europe,0,1,no-agent,yes,0,0,0.0,0.000000,0
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,Northern Europe,0,0,no-agent,yes,0,0,0.0,4.330733,0
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,Northern Europe,0,1,304.0,no,0,0,0.0,4.330733,0
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,Northern Europe,0,1,240.0,no,2,0,0.0,4.595120,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,City Hotel,0,23,2017,August,35,30,2,5,2,...,Western Europe,0,1,394.0,no,1,0,0.0,4.576153,0
119386,City Hotel,0,102,2017,August,35,31,2,5,3,...,Western Europe,0,1,9.0,no,3,0,0.0,5.422436,0
119387,City Hotel,0,34,2017,August,35,31,2,5,2,...,Western Europe,0,1,9.0,no,3,0,0.0,5.067079,0
119388,City Hotel,0,109,2017,August,35,31,2,5,2,...,Northern Europe,0,1,89.0,no,1,0,0.0,4.657763,0


In [57]:
# --- Remove old columns and useless columns  ---
# Variables to remove
cols_rm = ["company", #'sub_continent',
           "arrival_date_month",'arrival_date_week_number','arrival_date_day_of_month',"lead_time",
           'stays_in_week_nights','stays_in_weekend_nights', 'total_nights_bucket', #total_nights_bucekt can be replaced by total_nights
           'country', 'arrival_date',
           "previous_bookings_not_canceled", 
           "reserved_room_type",
           "assigned_room_type", 'distribution_channel',
           "agent", "agent_new", 'sub_continent',
           "days_in_waiting_list", "adr", "reservation_status", "reservation_status_date", "required_car_parking_spaces"]

# Drop columns
data_clean.drop(cols_rm, axis = 1, inplace = True)

In [61]:
# --- Fix data types --- 
# Categorical variables
categorical_vars = ["is_canceled", 
                    "arrival_month",# add total_nights_bucket if not removed
                    "meal", "market_segment", #"distribution_channel",
                    "continent", 'is_repeated_guest',
                    'domestic',
                    "got_room_booked",
                    "booked_by_agent",
                    "customer_type", "booked_by_company", "required_car_parking"]

# Convert to category type
data_clean[categorical_vars] = data_clean[categorical_vars].astype("category")

# Convert to numerical
data_clean['children'] = pd.to_numeric(data_clean['children'], downcast = 'integer')

# Check datatypes
data_clean.dtypes

hotel                          object
is_canceled                  category
arrival_date_year               int64
adults                          int64
children                         int8
babies                          int64
meal                         category
market_segment               category
is_repeated_guest            category
previous_cancellations          int64
booking_changes                 int64
deposit_type                   object
customer_type                category
total_of_special_requests       int64
arrival_month                category
log_lead_time                 float64
total_nights                    int64
domestic                     category
continent                    category
previous_bookings               int64
got_room_booked              category
booked_by_agent              category
agent_risk                      int64
booked_by_company            category
log_days_in_waiting_list      float64
log_adr                       float64
required_car

In [62]:
# Save clean data as csv 
data_clean.to_csv("../02_Data/hotel_bookings_clean.csv", index = False)

# Rename arrival_date_year to arrival_year
# Rename categories in dummies

In [63]:
# One-hot encoding the categorical variables
num = data_clean.describe().columns # column names of numerical columns
print(num)

Index(['arrival_date_year', 'adults', 'children', 'babies',
       'previous_cancellations', 'booking_changes',
       'total_of_special_requests', 'log_lead_time', 'total_nights',
       'previous_bookings', 'agent_risk', 'log_days_in_waiting_list',
       'log_adr'],
      dtype='object')


In [65]:
cat = data_clean.columns.difference(num) # column names of categorical columns
print(cat)

data_clean_dum_cat = pd.get_dummies(data_clean[cat])
data_clean_dum_cat.head(5)

Index(['arrival_month', 'booked_by_agent', 'booked_by_company', 'continent',
       'customer_type', 'deposit_type', 'domestic', 'got_room_booked', 'hotel',
       'is_canceled', 'is_repeated_guest', 'market_segment', 'meal',
       'required_car_parking'],
      dtype='object')


,arrival_month_1,arrival_month_2,arrival_month_3,arrival_month_4,arrival_month_5,arrival_month_6,arrival_month_7,arrival_month_8,arrival_month_9,arrival_month_10,...,market_segment_Offline TA/TO,market_segment_Online TA,market_segment_Undefined,meal_BB,meal_FB,meal_HB,meal_SC,meal_Undefined,required_car_parking_0,required_car_parking_1
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
4,0,0,0,0,0,0,1,0,0,0,...,0,1,0,1,0,0,0,0,1,0


In [68]:
# Merge the one-hot encoded categorical vars and the numerical vars
data_clean_ohe = data_clean[num].join(data_clean_dum_cat)
# Save as CSV file
data_clean_ohe.to_csv("../02_Data/hotel_bookings_ohe.csv", index = False)